In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/ashrae-energy-prediction/building_metadata.csv
/kaggle/input/ashrae-energy-prediction/weather_train.csv
/kaggle/input/ashrae-energy-prediction/train.csv
/kaggle/input/ashrae-energy-prediction/sample_submission.csv
/kaggle/input/ashrae-energy-prediction/test.csv
/kaggle/input/ashrae-energy-prediction/weather_test.csv


In [2]:
PATH = '/kaggle/input/ashrae-energy-prediction/train.csv'
df = pd.read_csv(PATH)
df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [3]:
PATH = '/kaggle/input/ashrae-energy-prediction/test.csv'
test = pd.read_csv(PATH)
test.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [4]:
time = 'timestamp'
target = 'meter_reading'
building = 'building_id'
meter = 'meter'

df[time] = pd.to_datetime(df[time])
test[time] = pd.to_datetime(test[time])

In [5]:
week = 'week'
hr = 'hour'
df[week] = df[time].dt.week
test[week] = test[time].dt.week
df[hr] = df[time].dt.hour
test[hr] = test[time].dt.hour

print(df.head())
print(test.head())

   building_id  meter  timestamp  meter_reading  week  hour
0            0      0 2016-01-01            0.0    53     0
1            1      0 2016-01-01            0.0    53     0
2            2      0 2016-01-01            0.0    53     0
3            3      0 2016-01-01            0.0    53     0
4            4      0 2016-01-01            0.0    53     0
   row_id  building_id  meter  timestamp  week  hour
0       0            0      0 2017-01-01    52     0
1       1            1      0 2017-01-01    52     0
2       2            2      0 2017-01-01    52     0
3       3            3      0 2017-01-01    52     0
4       4            4      0 2017-01-01    52     0


In [6]:
pred = 'prediction'
group = [building, meter, week, hr]
gp = df.groupby(group)[[target]].median().rename({target:pred},axis=1)
test = test.merge(gp, on=group, how='left')
test.head()

,row_id,building_id,meter,timestamp,week,hour,prediction
0,0,0,0,2017-01-01,52,0,224.90350
1,1,1,0,2017-01-01,52,0,92.69165
2,2,2,0,2017-01-01,52,0,5.52875
3,3,3,0,2017-01-01,52,0,170.29850
4,4,4,0,2017-01-01,52,0,1140.56000


In [7]:
PATH = '/kaggle/input/ashrae-energy-prediction/sample_submission.csv'
sub = pd.read_csv(PATH)
sub.head().append(sub.tail())

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
41697595,41697595,0
41697596,41697596,0
41697597,41697597,0
41697598,41697598,0
41697599,41697599,0


In [8]:
print(len(sub))
print(len(test))
print((sub['row_id'] == test['row_id']).all())

PATH = 'submission.csv.gz'
sub[target] = test[pred].ffill().bfill()
sub.to_csv(PATH, index=False)

41697600
41697600
True


In [9]:
sub.head().append(sub.tail())

,row_id,meter_reading
0,0,224.90350
1,1,92.69165
2,2,5.52875
3,3,170.29850
4,4,1140.56000
41697595,41697595,4.75000
41697596,41697596,5.90000
41697597,41697597,0.95000
41697598,41697598,172.42500
41697599,41697599,4.20000


In [10]:
sub.isna().any()

row_id           False
meter_reading    False
dtype: bool

In [11]:
np.isinf(sub).any()

row_id           False
meter_reading    False
dtype: bool